# **Demo: Langchain Model Prompt Outputparser**

# **Steps to Perform:**

1. Set up the Environment
2. Call Direct API to OpenAI
3. Call the Direct API at OpenAI
4. Use the Chat Model
5. Format a New Message
6. Generate a Response in a New Style
7. Output Parsers
8. Use the Output Parser

# **Step 1: Set up the Environment**



*   Install the required packages and configure the environment, including OpenAI and LangChain, for making API calls.
*   Import essential modules and set up the OpenAI API key for making direct API calls to OpenAI through langchain.




In [1]:
import os
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# **Step 2: Call the Direct API at OpenAI**



*   Create a function called **get_completion** that receives a prompt and model as input and provides a completion as output.
*   Verify the functionality of the function by asking a straightforward question, such as **What is 1+1?**.

In [2]:
client = openai.OpenAI()

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

In [4]:
print(get_completion("What is 1+1?"))

1+1 equals 2.


# **Step 3: Call API Through LangChain**



*   Use LangChain by creating an instance of the ChatOpenAI class.
*   Modify parameters like **customer_style** and **customer_email** to influence the tone and formality of the generated responses.




In [5]:
# Start by creating an instance of the ChatOpenAI class.
chat = ChatOpenAI(temperature=0.0)

# Define a template for our prompts.
template_string = """Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```"""
prompt_template = ChatPromptTemplate.from_template(template_string)

# Use this template to format our messages.
customer_style = "American English in a casual tone"
customer_email = """
I'm super excited about the new gaming console I bought! It arrived in just 2 days and I've been playing non-stop. Totally worth the price!
"""

customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

/var/folders/gn/phv95k851gl8rtg00v3syx2r0000gn/T/ipykernel_57269/1326818511.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat = ChatOpenAI(temperature=0.0)


# **Step 4: Use the Chat Model**




*   Use the ChatOpenAI instance to produce a response in the designated style, aligning with the formatted messages.




In [6]:
# Use our ChatOpenAI instance to generate a response.
customer_response = chat(customer_messages)
print(customer_response.content)

/var/folders/gn/phv95k851gl8rtg00v3syx2r0000gn/T/ipykernel_57269/2266241821.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


I'm so pumped about the new gaming console I got! It came in just 2 days and I've been playing non-stop. Definitely worth the money!


In [8]:
# repeat the above step with invoke method
# customer_response = chat.invoke(customer_messages)
# print(customer_response.content)

# **Step 5: Format a New Message**


*   Format the newly created message in the designated style, preparing it for interaction with the chat model.




In [9]:
# Format a new message using a different style.
service_reply = "Hey there, we're glad you're enjoying your new gaming console. Game on!"
service_style_pirate = "a cheerful tone that speaks in English Pirate"
service_messages = prompt_template.format_messages(style=service_style_pirate, text=service_reply)
print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a cheerful tone that speaks in English Pirate. text: ```Hey there, we're glad you're enjoying your new gaming console. Game on!```


# **Step 6: Generate a Response in a New Style**


*   Retrieve the generated response, in this case, would be in the manner of a polite English Pirate.




In [10]:
# Generate a response in our new style using the ChatOpenAI instance.
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, me hearties! We be pleased as a parrot in a treasure chest to hear ye be enjoyin' yer new gaming console. Sail on, me matey! Arrr!


# **Step 7: Output Parsers**


*   Clearly outline the preferred format for the language model's output using an output parser.






In [11]:
# Define how we would like the output from the language model to look like.
desired_output = {
    "gift": False,
    "delivery_days": 2,
    "price_value": "Totally worth the price!"
}

# Create a customer review and a template for extracting information from the review.
customer_review = """\
I'm super excited about the new gaming console I bought! It arrived in just 2 days and I've been playing non-stop. Totally worth the price!
"""

review_template = """\
For the following text, extract the following information:
gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.
delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.
price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.
Format the output as JSON with the following keys:
gift
delivery_days
price_value
text: {text}
"""

# **Step 8: Use the Output Parser**


*   Use the defined schemas to instantiate the **StructuredOutputParser** class, forming an instance named **output_parser**.






In [12]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

gift_schema = ResponseSchema(name="gift",
description="Was the product given as a present to someone? \
Answer True if yes, False if not or unknown.")

delivery_days_schema = ResponseSchema(name="delivery_days",
description="How many days was the delivery period for the product? \
If this information is not found, output -1.")

price_value_schema = ResponseSchema(name="price_value",
description="Extract any sentences about the cost or value of the product, \
and output them as a comma separated Python list.")

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

You can use these format instructions to create a new prompt, generate a response, and parse the output.

In [13]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [14]:
format_instructions = output_parser.get_format_instructions()

In [15]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the product given as a present to someone? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days was the delivery period for the product? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the cost or value of the product, and output them as a comma separated Python list.
}
```


In [16]:
prompt = ChatPromptTemplate.from_template(template=review_template)
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)

response = chat(messages)
output_dict = output_parser.parse(response.content)

In [17]:
print(output_dict)

{'gift': False, 'delivery_days': 2, 'price_value': ['Totally worth the price!'], 'text': "I'm super excited about the new gaming console I bought! It arrived in just 2 days and I've been playing non-stop. Totally worth the price!"}


In [18]:
print(output_dict.get('text'))

I'm super excited about the new gaming console I bought! It arrived in just 2 days and I've been playing non-stop. Totally worth the price!


# **Conclusion:**
In this activity, you successfully configured the environment, utilized OpenAI's Direct API, interacted with the Chat Model, formatted messages, generated responses, and employed output parsers. The process was efficient and demonstrated effective communication with the OpenAI models.